In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [89]:
FOOD = '/data/share/bdm/nyc_restaurants.csv'

In [90]:
food = sc.textFile(FOOD, use_unicode=False).cache()
list(enumerate(food.first().split(',')))

[(0, 'CAMIS'),
 (1, 'DBA'),
 (2, 'BORO'),
 (3, 'BUILDING'),
 (4, 'STREET'),
 (5, 'ZIPCODE'),
 (6, 'PHONE'),
 (7, 'CUISINE DESCRIPTION'),
 (8, 'INSPECTION DATE'),
 (9, 'ACTION'),
 (10, 'VIOLATION CODE'),
 (11, 'VIOLATION DESCRIPTION'),
 (12, 'CRITICAL FLAG'),
 (13, 'SCORE'),
 (14, 'GRADE'),
 (15, 'GRADE DATE'),
 (16, 'RECORD DATE'),
 (17, 'INSPECTION TYPE')]

In [91]:
noHeaderRDD = food.filter(lambda x: not x.startswith('CUISINE'))
print (food.first())
print (noHeaderRDD.first())

CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE
CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE


In [92]:
def getCuisine(partId, list_of_rest):
    count = 0
    if partId == 0:
        list_of_rest.next()
    import csv
    reader = csv.reader(list_of_rest)
    for row in reader:
        if row[14]!= 'F':
            (cuisine) = (row[7])
            count = count + 1
            yield (cuisine)

cuisine = food.mapPartitionsWithIndex(getCuisine)
cuisine.take(11)

['Chicken',
 'Jewish/Kosher',
 'Caribbean',
 'Italian',
 'Korean',
 'Donuts',
 'Sandwiches/Salads/Mixed Buffet',
 'American',
 'American',
 'Juice, Smoothies, Fruit Salads',
 'Pizza/Italian']

In [93]:
cc = cuisine.flatMap(lambda line: line.split()) \
        .map(lambda x: (x.lower(), 1)) \
        .groupByKey() \
        .mapValues(lambda values: sum(values))
cc.top(25, key=lambda x: x[1])
#sorted_cc = sorted(cc, key=lambda tup: tup[1])

[('american', 25),
 ('chinese', 6),
 ('(cuban,', 6),
 ('american)', 6),
 ('dominican,', 6),
 ('central', 6),
 ('south', 6),
 ('&', 6),
 ('rican,', 6),
 ('puerto', 6),
 ('pizza', 6),
 ('latin', 6),
 ('bakery', 5),
 ('japanese', 5),
 ('hamburgers', 4),
 ('delicatessen', 3),
 ('salads', 3),
 ('caribbean', 3),
 ('tex-mex', 3),
 ('caf\xc3\x83\xc2\xa9/coffee/tea', 3),
 ('mexican', 2),
 ('juice,', 2),
 ('chicken', 2),
 ('buffet', 2),
 ('seafood', 2)]